In [1]:
#1. Установка необходимых библиотек (если ещё не установлены)
!pip install transformers datasets peft bitsandbytes torch accelerate sentencepiece pandas psutil rich colorama


In [2]:
#2. Импорт библиотек
import os
import torch
import pandas as pd
import psutil
import glob
import re
from datasets import Dataset
from transformers import (
    AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer, 
    DataCollatorForSeq2Seq, BitsAndBytesConfig
)
from peft import get_peft_model, LoraConfig, TaskType

# 🔹 Отключаем предупреждение про многопоточность `tokenizers`
os.environ["TOKENIZERS_PARALLELISM"] = "false"


<b>Функция для расчёта оптимального размера чанка<b/>

In [3]:
def get_optimal_chunk_size(min_chunk_size=10000, max_chunk_size=50000):
    """
    Определяет оптимальный размер чанка на основе доступной RAM и VRAM.
    """
    total_ram = psutil.virtual_memory().available / (1024 ** 3)
    total_vram = torch.cuda.get_device_properties(0).total_memory / (1024 ** 3) if torch.cuda.is_available() else 0
    estimated_chunk_size = int((total_ram + total_vram) * 2500)
    return max(min(estimated_chunk_size, max_chunk_size), min_chunk_size)


<b>Загрузка и обработка датасета<b/>

In [4]:
df = pd.read_csv("output_final_4persent.csv", sep=";").dropna()

# Исправляем названия столбцов, если есть ошибки
if "text_wich_errors" in df.columns:
    df.rename(columns={"text_wich_errors": "text_with_errors"}, inplace=True)


<b>Создание папки для чекпоинтов и чанков<b/>

In [5]:
checkpoint_dir = "./t5-spell-corrector-v2"  # Папка для новых чекпоинтов
os.makedirs(checkpoint_dir, exist_ok=True)  # Создаём папку, если её нет

chunk_size = get_optimal_chunk_size()
chunk_files = sorted(glob.glob("chunk_v2_*.csv"))  # Проверяем, есть ли уже чанки

# Если чанков нет, создаём заново
if chunk_files:
    print("✅ Чанки уже существуют, повторное создание не требуется.")
else:
    print("🔹 Чанки не найдены, создаем заново...")
    for i, start in enumerate(range(0, len(df), chunk_size)):
        df.iloc[start:start + chunk_size].to_csv(f"chunk_v2_{i + 1}.csv", sep=";", index=False)
    print(f"✅ Датасет разбит на {i + 1} новых чанков.")


✅ Чанки уже существуют, повторное создание не требуется.


<b>Загрузка токенизатора и модели<b/>

In [6]:
model_name = "UrukHan/t5-russian-spell"
tokenizer = AutoTokenizer.from_pretrained(model_name)
quantization_config = BitsAndBytesConfig(load_in_8bit=True)  # Используем 8-битное квантование

model = AutoModelForSeq2SeqLM.from_pretrained(model_name, quantization_config=quantization_config, device_map="auto")
model.config.use_cache = False  # Отключаем кеширование


<b>Функция токенизации данных<b/>

In [7]:
def preprocess_function(examples):
    model_inputs = tokenizer(examples["input_text"], padding="max_length", truncation=True, max_length=256)
    labels = tokenizer(examples["target_text"], padding="max_length", truncation=True, max_length=256)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


<b>Настройка адаптивного обучения LoRA<b/>

In [8]:
lora_config = LoraConfig(task_type=TaskType.SEQ_2_SEQ_LM, r=8, lora_alpha=32, lora_dropout=0.1)
model = get_peft_model(model, lora_config)

# Включаем градиенты только для LoRA-адаптеров (экономия памяти)
for name, param in model.named_parameters():
    if "lora" in name:
        param.requires_grad = True
    else:
        param.requires_grad = False


<b>Параметры обучения<b/>

In [9]:
training_args = TrainingArguments(
    output_dir=checkpoint_dir,  # Чекпоинты сохраняются в новой папке
    eval_strategy="steps",
    save_strategy="steps",
    save_steps=100,  # Сохраняем каждые 100 шагов
    logging_steps=100,
    learning_rate=5e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=16,
    weight_decay=0.01,
    fp16=True,
    optim="adamw_bnb_8bit",
    label_names=["labels"],
)


<b>Функция поиска последнего чекпоинта<b/>

In [10]:
def find_last_valid_checkpoint():
    checkpoint_list = sorted(
        glob.glob(f"{checkpoint_dir}/checkpoint-*"),
        key=lambda x: int(re.search(r'checkpoint-(\d+)', x).group(1)) if re.search(r'checkpoint-(\d+)', x) else 0
    )
    for checkpoint in reversed(checkpoint_list):
        if os.path.exists(os.path.join(checkpoint, "trainer_state.json")):
            return checkpoint  
    return None  

last_checkpoint = find_last_valid_checkpoint()

if last_checkpoint:
    print(f"✅ Продолжаем обучение с последнего рабочего чекпоинта: {last_checkpoint}")
else:
    print("🚀 Обучение начинается с нуля.")


✅ Продолжаем обучение с последнего рабочего чекпоинта: ./t5-spell-corrector-v2/checkpoint-3700


<b>Обучение на чанках<b/>

In [ ]:
chunk_files = sorted(glob.glob("chunk_v2_*.csv"))

for i, chunk_file in enumerate(chunk_files):
    print(f"▶️ Обучение на новом чанке {i+1}/{len(chunk_files)}: {chunk_file}")
    
    # Загружаем данные
    df_chunk = pd.read_csv(chunk_file, sep=";")
    dataset = Dataset.from_pandas(df_chunk).train_test_split(test_size=0.1, seed=42)

    # Токенизируем данные
    dataset = dataset.map(lambda x: {"input_text": "Исправь текст: " + x["text_with_errors"], "target_text": x["corrected_text"]})
    dataset = dataset.map(preprocess_function, batched=True)

    if "input_text" in dataset["train"].column_names and "target_text" in dataset["train"].column_names:
        dataset = dataset.remove_columns(["input_text", "target_text"])

    # Создаём Trainer
    trainer = Trainer(model=model, args=training_args, train_dataset=dataset["train"], eval_dataset=dataset["test"], data_collator=DataCollatorForSeq2Seq(tokenizer, model=model))

    # Начинаем обучение
    if last_checkpoint:
        print(f"🔄 Продолжаем обучение с {last_checkpoint}")
        trainer.train(resume_from_checkpoint=last_checkpoint)
    else:
        print("🚀 Начинаем обучение с нуля...")
        trainer.train()

    # Сохраняем чекпоинт
new_checkpoint = f"{checkpoint_dir}/checkpoint-{i+1}"
trainer.save_model(new_checkpoint)

# 🔹 Очистка кеша GPU
torch.cuda.empty_cache()
gc.collect()

# Обновляем last_checkpoint
last_checkpoint = new_checkpoint


print("🎉 Обучение завершено! Новые чекпоинты сохранены в:", checkpoint_dir)


▶️ Обучение на новом чанке 1/5: chunk_v2_1.csv


Map:   0%|          | 0/45000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/45000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

🔄 Продолжаем обучение с ./t5-spell-corrector-v2/checkpoint-3700


Step,Training Loss,Validation Loss
3800,0.033500,0.023563
3900,0.034900,0.023476
4000,0.032500,0.023346
4100,0.032800,0.023295
4200,0.033800,0.023198
4300,0.032900,0.023101
4400,0.034100,0.023018
4500,0.031800,0.022916
4600,0.031400,0.022855
4700,0.032000,0.022786
